In [2]:
# установка pycuda в GoogleColab
!pip install pycuda

     |████████████████████████████████| 1.6MB 13.2MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 81kB 13.1MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621013 sha256=d88664266ef1af1adbbd1510edf93c06260c0b4e737151601316b4ac37f4dd9c
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=2d30d38a4842b8872740b263a395d99af60761a47e13f3fd27f44fb53f934816
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [27]:
# импорт библиотек, необходимых для работы

import numpy as np
import time

import pycuda.gpuarray as gpuarray
import pycuda.autoinit
from pycuda.tools import make_default_context
from  pycuda import  driver
from pycuda.compiler import SourceModule

In [28]:
make_default_context().get_device().name()

'Tesla T4'

In [29]:
# перенос GPU алгоритма с плюсов в питон с помощью 'SourceModule'

calc_gpu = SourceModule("""
                __global__ void Calc(double *x, double *y, double *count) {
        int idx = blockIdx.x * blockDim.x + threadIdx.x; 
        int threadCount = gridDim.x * blockDim.x;
        int N = 65536*16;
        int count_gpu = 0;
        for (int i = idx; i < N; i += threadCount) {
                if (x[i] * x[i] + y[i] * y[i] < 1) {
                        count_gpu++;
                }
        }
        atomicAdd(count , count_gpu);
}
""")

In [33]:
# CPU алгоритм

def calc_cpu(N):
    count_cpu = 0
    for i in range(N):
        if x[i] ** 2 + y[i] ** 2 < 1:
            count_cpu += 1
    return count_cpu*4/N

In [34]:
N = 65536*16
x, y = np.random.random(N), np.random.random(N)
block_size = (256, 1, 1)
grid_size = (int(N / (128 * block_size[0])), 1)
count = np.zeros(1)
calc = calc_gpu.get_function("Calc")

In [35]:
start_cpu = time.time()
pi_cpu = calc_cpu(N)
end_cpu = time.time()

In [36]:
start = time.time()
calc(driver.In(x), driver.In(y),driver.Out(count), block = block_size, grid = grid_size)
driver.Context.synchronize()
end = time.time()

In [38]:
print('Время на GPU {}'.format(end - start), '\nВремя на  CPU {}'.format(end_cpu - start_cpu))
print('Число pi на GPU для сравнения ', float(count*4/N), '\nЧисло pi на CPU для сравнения ', pi_cpu)
print('Множитель ускорения {}'.format((end_cpu - start_cpu)/ (end - start)))
print('Сравнение с числом pi: ')
print('GPU:', abs(np.pi -  float(count*4/N)) )
print('CPU:', abs(np.pi -  pi_cpu) )

Время на GPU 0.012803792953491211 
Время на  CPU 1.2068207263946533
Число pi на GPU для сравнения  3.1400413513183594 
Число pi на CPU для сравнения  3.1400413513183594
Множитель ускорения 94.25493920265163
Сравнение с числом pi: 
GPU: 0.001551302271433741
CPU: 0.001551302271433741
